In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129431")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129431
Azure region: southcentralus
Subscription id: f39cb977-6a3a-445b-a26a-b9a791c5fd89
Resource group: aml-quickstarts-129431


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "D2V2Cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import normal
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "learning_rate": normal(10,3),
        "keep_probability": uniform(0.05,0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)

# Specify a Policy
# If, after 10 intervals, the current run has an absolute distance of > 0.2 from the current best run, terminate it.
policy = BanditPolicy(evaluation_interval=10,
                      slack_amount=0.2,
                      delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("./training", compute_target=cpu_cluster, entry_script="train.py", use_docker=False)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
                                     estimator=est,
                                     policy=policy,
                                     primary_metric_name="accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

WARNING - 'use_docker' parameter will be deprecated. Please use 'environment_definition' instead.


In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_46aad9f3-0fd5-48e9-ad06-ce03fa7eadaf
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_46aad9f3-0fd5-48e9-ad06-ce03fa7eadaf?wsid=/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourcegroups/aml-quickstarts-129431/workspaces/quick-starts-ws-129431

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-06T06:36:11.534525][API][INFO]Experiment created<END>\n""<START>[2020-12-06T06:36:12.385102][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-06T06:36:12.677909][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-06T06:36:13.4664620Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_46aad9f3-0fd5-48e9-ad06-ce03fa7eadaf
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_46aad9f3-0fd5-48e9-ad06-ce03fa7eadaf?wsid=/subscriptions/f39cb

{'runId': 'HD_46aad9f3-0fd5-48e9-ad06-ce03fa7eadaf',
 'target': 'D2V2Cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-12-06T06:36:11.329407Z',
 'endTimeUtc': '2020-12-06T06:42:21.580629Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "ac

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###